In [64]:
import numpy as np
from scipy.stats import qmc
import random
from docplex.mp.model import Model

In [65]:
def sobel_random_d():
    sobol = qmc.Sobol(d=7, scramble=True)
    
    scenarios = sobol.random(n=100)
    
    scenarios_h1 = scenarios * 40
    
    scenarios_h1 = scenarios_h1.astype(int)
    
    scenarios = sobol.random(n=100)
    
    scenarios_h2 = scenarios * 30 #hospital 2 is smaller
    scenarios_h2 = scenarios_h2.astype(int)
    
    D = {(t+1, 1, xi+1): scenarios_h1[xi,t] for t in range(0, num_time_periods) for xi in range(0, num_scenarios)}
    D.update({(t+1, 2, xi+1): scenarios_h2[xi,t] for t in range(0, num_time_periods) for xi in range(0, num_scenarios)})


    return D


In [86]:
T = range(1, 7 + 1)  # Time horizon
N = range(1, 2 + 1)  # Hospitals
M = range(1, 3 + 1)  # Remaining shelf life
scenarios = range(1, 100 + 1)  # Remaining shelf life


num_hospitals = len(N)
num_shelf_life = len(M)
num_time_periods = len(T)
num_scenarios = len(scenarios)

def model_(B,D):    
    model = Model(name='BloodTransshipment')
    
    #####################
    
    Y = {(i): model.integer_var(name=f'orders_first_stage_{i}') for i in N} #ORDERS FOR HOS i for first stage
    Y.update({(i, t, xi): model.integer_var(name=f'orders_second_stage_to_{i}_time_{t}__senario_{xi}') for i in N for t in T[1:] for xi in scenarios})
    
    
    X = {(i, j, m): model.integer_var(name=f'Trans_first_stage_from_{i}to_{j}age{m}') for i in N for j in N if i!=j for m in M } #trans ORDERSs From HOS i to j for first stage
    
    f = {(i, t, xi): model.integer_var(name=f'f_{i}_{t}_{xi}') for i in N for t in T for xi in scenarios} #shortage
    o = {(i, t, xi): model.integer_var(name=f'o_{i}_{t}_{xi}') for i in N for t in T for xi in scenarios} #outdated
    v = {(i, t, xi): model.integer_var(name=f'v_{i}_{t}_{xi}') for i in N for t in T for xi in scenarios} #inventory
    
    
    i_s = {(i,m,1): model.integer_var(name=f'is_{i}_{m}_{1}') for i in N for m in M}
    i_s.update({(i, m, t, xi): model.integer_var(name=f'is_{i}_{t}_{m}_{xi}') for i in N for t in T if t!=1 for m in M for xi in scenarios})
    
    
    i_e = {(i, m, t, xi): model.integer_var(name=f'ie_{i}_{t}_{m}_{xi}') for i in N for t in T for m in M for xi in scenarios}
    
    s = {(i): model.integer_var(name=f'TargetInventory_{i}') for i in N }
    
    a = {(i, m, t, xi): model.integer_var(name=f'a_{i}_{t}_{m}_{xi}') for i in N for t in T for m in M for xi in scenarios}
    
    
    G = 16  # Emergency order cost
    H = 1   #
    E = 13 # Expiry cost
    R = 1  # Ordering cost
    C = 1.5 # Transshipment cost
    P = 0.5  # Scenario probabilities
    
    
    # objective function
    model.minimize(model.sum(R * Y[i] for i in N) + model.sum(C * X[i, j, m] for i in N for j in N if i != j for m in M)#STAGE ONE
              + model.sum(model.sum(H * v[i, 1, xi] + E * o[i, 1, xi] + G * f[i, 1, xi]) for i in N for xi in scenarios)
              + model.sum(R * Y[i, t, xi] + H * v[i, t, xi] + E * o[i, t, xi] + G * f[i, t, xi] for t in T[1:] for i in N for xi in scenarios))#STAGE TWO
    
    
    
    # nsures blood units balance in the first stage at each hospital
    for i in N:
      for m in M:
          for xi in scenarios:
              model.add_constraint(
                  model.sum(i_s[i,m,1]) + model.sum(X[j, i, m] for j in N if i != j) == model.sum(X[i, j, m] for j in N if i != j) + model.sum(a[i, m, 1, xi]) + model.sum(i_e[i, m, 1, xi]),
                  ctname=f"stage_one_transshipment_{i}_{m}_{xi}"
              )
    
    
    # Ensures blood units balance in the second stage at each hospital.
    for i in N:
      for m in M:
          for xi in scenarios:
              model.add_constraint(
                  model.sum(i_s[i,m,t,xi] for t in T[1:]) == model.sum(a[i,m,t,xi] + i_e[i,m,t,xi] for t in T[1:]) ,
                  ctname=f"inv_{i}_{m}_{xi}"
              )
    
    #Sets the initial blood units at each hospital.
    for i in N:
      for m in M:
          model.add_constraint( model.sum(i_s[i,m,1] == model.sum(B[i,m])) )
    
    
    #Makes sure each hospital uses blood units equal to its demand.
    for i in N:
      for t in T:
          for xi in scenarios:
              model.add_constraint(model.sum(a[i,m,t,xi] for m in M) + model.sum(f[i,t,xi]) == model.sum(D[t,i,xi]))
    
    for i in N:
      for t in T:
          for xi in scenarios:
    
              #Calculates the inventory level excluding the blood units with one day of shelf life left.
              model.add_constraint(v[i,t,xi] == model.sum(i_e[i,m,t,xi] for m in M if m!=1))
              #Calculates the amount of blood units that become outdated.
              model.add_constraint(o[i,t,xi] == model.sum(i_e[i,1,t,xi]) )
    
    
    
    for i in N:
      for m in M[:-1]:
          for xi in scenarios:
              for t in T[:-1]:
                 #ensures the leftover blood units are equal to the received units in the next time period.
                  model.add_constraint(model.sum(i_e[i,m+1,t,xi]) == model.sum(i_s[i,m,t+1,xi]))
    
    
    
    #Matches the ordered blood units with the received units in the next stage
    for i in N:
      for xi in scenarios:
          model.add_constraint(model.sum(i_s[i, len(M) , 2, xi]) == model.sum(Y[i]))
    
    
    for i in N:
      for t in T[1:1]:
          for xi in scenarios:
              model.add_constraint(model.sum(i_s[i , len(M) , t+1 , xi]) == model.sum(Y[i,t,xi]))
    
    #defines the target inventory level at each hospital
    for i in N:
      for t in T[1:]:
          for xi in scenarios:
              model.add_constraint(model.sum(s[i]) - model.sum(i_s[i,m,t,xi] for m in M) == model.sum(Y[i,t,xi]))
    
    
    model.solve()

    return model , X , Y



In [181]:
B = {(i, m): random.randint(0, 20) for i in range(1, num_hospitals + 1) for m in range(1, num_shelf_life + 1)}
B

{(1, 1): 20, (1, 2): 4, (1, 3): 5, (2, 1): 9, (2, 2): 0, (2, 3): 4}

In [182]:
from tqdm import tqdm

invs = {}
ords = {}
trans = {}

for i in tqdm(range(10)):    
    model , X , Y = model_(B,sobel_random_d())
    
    invs[i+1] = B.copy()
    ords[i+1]= Y.copy()
    trans[i+1] = X.copy()
    
    for i in N:
        for j in N:
            for m in M:
                if i!=j:
                    B[i,m] -= int(X[i,j,m].solution_value)
                    B[j,m] += int(X[i,j,m].solution_value)


    for i in N:
        B[i,1] = 0
        B[i,2] = B[i,3]
        B[i,3] = int(Y[i].solution_value)

    
    

    

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:08<00:00,  1.12it/s]


import numpy as np
from scipy.stats import nbinom

def generate_zinb(pi, r, p, size=1):
    # Generate a binary array with 'size' elements
    # An element is 1 with probability 'pi' and 0 with probability '1-pi'
    binary = np.random.choice([0, 1], size=size, p=[1-pi, pi])

    # Generate a negative binomial array with 'size' elements
    nb = nbinom.rvs(n=r, p=p, size=size)

    # Return the element-wise product of the binary and negative binomial arrays
    return binary * nb

# Generate samples from the ZINB distributions
zinb1 = generate_zinb(pi=0.6, r=4, p=0.6, size=1000)
zinb2 = generate_zinb(pi=0.6, r=3, p=0.57, size=1000)
zinb3 = generate_zinb(pi=0.25, r=15, p=0.57, size=1000)
zinb4 = generate_zinb(pi=0.25, r=15, p=0.48, size=1000)

# Print the first 10 samples from each distribution
print("ZINB(π = 0.6, r = 4, p = 0.6):", zinb1[:10])
print("ZINB(π = 0.6, r = 3, p = 0.57):", zinb2[:10])
print("ZINB(π = 0.25, r = 15, p = 0.57):", zinb3[:10])
print("ZINB(π = 0.25, r = 15, p = 0.48):", zinb4[:10])
